In [316]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import warnings
import itertools
import os
warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt

In [317]:
from gseapy.parser import Biomart

bm = Biomart(verbose=False, host="europe.ensembl.org")
## view validated marts
marts = bm.get_marts()
## view validated dataset
datasets = bm.get_datasets(mart='ENSEMBL_MART_ENSEMBL')
## view validated attributes
attrs = bm.get_attributes(dataset='hsapiens_gene_ensembl')
## view validated filters
filters = bm.get_filters(dataset='hsapiens_gene_ensembl')

In [318]:
marts

,Name,Description
0,ENSEMBL_MART_ENSEMBL,Ensembl Genes 103
1,ENSEMBL_MART_MOUSE,Mouse strains 103
2,ENSEMBL_MART_SEQUENCE,Sequence
3,ENSEMBL_MART_ONTOLOGY,Ontology
4,ENSEMBL_MART_GENOMIC,Genomic features 103
5,ENSEMBL_MART_SNP,Ensembl Variation 103
6,ENSEMBL_MART_FUNCGEN,Ensembl Regulation 103


### Add the names of the pathway for the DKO drugs


In [319]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [320]:
# Read Enrichr DKO enrichment
dko_enrichr = pd.read_csv('KO_data/DKO_Lung_Studies_Enrichment.csv',index_col=0)
dko_enrichr.Genes = dko_enrichr.Genes.apply(lambda x:x.split(';'))
dko_enrichr_ = explode(dko_enrichr,['Genes'], fill_value='')
dko_enrichr_ = dko_enrichr_.loc[:,['Genes','Term']].drop_duplicates().reset_index(drop=True)
dko_enrichr.head()

,Term,Genes,P-value,Overlap
0,Purine metabolism,"[RRM1, RRM2, NME4, PAICS, PFAS, PNP, ATIC, RRM...",1.790315e-16,13/129
1,Citrate cycle (TCA cycle),"[FH, SUCLG2, SDHC, SUCLG1, SDHD, SDHA, SDHB, DLD]",3.866059e-14,8/30
2,Pyrimidine metabolism,"[DTYMK, RRM1, PNP, RRM2, RRM2B, DPYD, CMPK1, N...",1.020544e-11,8/57
3,Pentose phosphate pathway,"[RPIA, GPI, PGM2, PGD, DERA]",3.832756e-08,5/30
4,Drug metabolism,"[RRM1, RRM2, RRM2B, DPYD, CMPK1, NME4, HPRT1]",5.304212e-08,7/108


In [321]:
# Read RECON3 pwthways & rules to do enrichment for DKO drugs
recon_rules = pd.read_csv('RECON3_subSystems_grRules_genes.csv')
recon_rules.gene = recon_rules.gene.apply(lambda x:str(str(x).split('.')[0]))
recon_rules.tail(5)

,Pathway,grRules,gene
14482,Oxidative phosphorylation,1351.1 and 1347.1 and 1329.1 and 1327.1 and 34...,1340
14483,Oxidative phosphorylation,(7384.1 and 7388.1 and 4519.1 and 10975.1 and ...,1340
14484,Oxidative phosphorylation,1351.1 and 1347.1 and 1329.1 and 1327.1 and 34...,84701
14485,Oxidative phosphorylation,1351.1 and 1347.1 and 1329.1 and 1327.1 and 34...,1337
14486,Oxidative phosphorylation,(7384.1 and 7388.1 and 4519.1 and 10975.1 and ...,1337


In [322]:
recon_rules.head(5)

,Pathway,grRules,gene
0,Beta-Alanine metabolism,8639.1 or 26.1 or 314.2 or 314.1,8639
1,Tyrosine metabolism,4129.1 or 8639.1 or 26.1 or 314.2 or 314.1 or ...,8639
2,"Glycine, serine, alanine, and threonine metabo...",8639.1,8639
3,Tyrosine metabolism,8639.1 or 26.1 or 314.2 or 314.1,8639
4,Histidine metabolism,26.1 or 314.1 or 8639.1 or 314.2,8639


In [323]:
gene_list = recon_rules.gene.unique().tolist()
results = bm.query(dataset='hsapiens_gene_ensembl', attributes=['entrezgene_id','hgnc_symbol'],#, 'go_id'],
                      filters={'entrezgene_id': gene_list})
results =  results.dropna().drop_duplicates().reset_index(drop=True)

In [324]:
recon_rules.shape

(14487, 3)

In [325]:
results.shape

(1872, 2)

In [326]:
len(gene_list)

1884

In [327]:
recon_rules['Genes'] = 0
for i in range(recon_rules.shape[0]):
    entrez = recon_rules.iloc[i,2]
    if int(entrez) in results.entrezgene_id.values:
        gene = set(results.loc[results.entrezgene_id == int(entrez),'hgnc_symbol'].values.tolist())
    else:
        gene = np.NAN
    recon_rules.iloc[i,3] =  gene


In [328]:
recon_rules.head(5)

,Pathway,grRules,gene,Genes
0,Beta-Alanine metabolism,8639.1 or 26.1 or 314.2 or 314.1,8639,AOC3
1,Tyrosine metabolism,4129.1 or 8639.1 or 26.1 or 314.2 or 314.1 or ...,8639,{AOC3}
2,"Glycine, serine, alanine, and threonine metabo...",8639.1,8639,{AOC3}
3,Tyrosine metabolism,8639.1 or 26.1 or 314.2 or 314.1,8639,{AOC3}
4,Histidine metabolism,26.1 or 314.1 or 8639.1 or 314.2,8639,{AOC3}


In [329]:
recon_rules = recon_rules.loc[:,['Pathway','Genes']].dropna()

In [330]:
recon_rules.Genes = recon_rules.Genes.apply(lambda x:''.join(x))

In [331]:
recon_rules.head()

,Pathway,Genes
0,Beta-Alanine metabolism,AOC3
1,Tyrosine metabolism,AOC3
2,"Glycine, serine, alanine, and threonine metabo...",AOC3
3,Tyrosine metabolism,AOC3
4,Histidine metabolism,AOC3


In [332]:
recon_rules.shape

(14417, 2)

In [333]:
# Add the names of the pathway for the DKO drugs
dko_one_drug = pd.read_csv('KO_data/DKO_Drugs_OneDrug.csv',index_col=0)
dko_one_drug.Gene_Pair = dko_one_drug.Gene_Pair.apply(lambda x:str(x).replace('{','').replace('}','').replace(', ',';').replace("'",''))
dko_one_drug.Gene_Pair = dko_one_drug.Gene_Pair.apply(lambda x:x.split(';'))
dko_one_drug_ =  explode(dko_one_drug,['Gene_Pair'], fill_value='')
dko_one_drug_.head()

,Drug,Number_Genes,Phenotype_Essentiality,Phenotype_Safety,Gene_Pair
0,Gemcitabine,3,1.66,1.0,CMPK1
1,Gemcitabine,3,1.66,1.0,SLC29A2
2,Gemcitabine,3,1.66,1.0,TYMS
3,Gemcitabine,3,1.66,1.0,SLC29A2
4,Gemcitabine,3,1.66,1.0,SLC29A2


In [334]:
genes =  dko_one_drug_.Gene_Pair.unique()
dko_one_drug_['Pathways'] = 0
for gene in genes:
    gene_pathways = recon_rules.loc[recon_rules.Genes ==gene,'Pathway'].values.tolist()
    dko_one_drug_.loc[dko_one_drug_.Gene_Pair ==gene,'Pathways'] = ';'.join(gene_pathways)
dko_one_drug_.head()

,Drug,Number_Genes,Phenotype_Essentiality,Phenotype_Safety,Gene_Pair,Pathways
0,Gemcitabine,3,1.66,1.0,CMPK1,Nucleotide interconversion;Nucleotide intercon...
1,Gemcitabine,3,1.66,1.0,SLC29A2,"Transport, extracellular;Transport, extracellu..."
2,Gemcitabine,3,1.66,1.0,TYMS,Nucleotide interconversion
3,Gemcitabine,3,1.66,1.0,SLC29A2,"Transport, extracellular;Transport, extracellu..."
4,Gemcitabine,3,1.66,1.0,SLC29A2,"Transport, extracellular;Transport, extracellu..."


In [335]:
dko_one_drug_path = dko_one_drug_.loc[dko_one_drug_.Pathways.apply(lambda x:len(x))>0,:]
dko_one_drug_path = dko_one_drug_path.drop_duplicates().reset_index(drop=True)
dko_one_drug_path.to_csv('KO_data/DKO_Drugs_OneDrug_Pathways.csv')
dko_one_drug_path.head(5)

,Drug,Number_Genes,Phenotype_Essentiality,Phenotype_Safety,Gene_Pair,Pathways
0,Gemcitabine,3,1.66,1.0,CMPK1,Nucleotide interconversion;Nucleotide intercon...
1,Gemcitabine,3,1.66,1.0,SLC29A2,"Transport, extracellular;Transport, extracellu..."
2,Gemcitabine,3,1.66,1.0,TYMS,Nucleotide interconversion
3,Gemcitabine,3,1.66,1.0,SLC29A1,"Transport, extracellular;Transport, mitochondr..."
4,Trifluridine,3,1.33,1.0,TYMS,Nucleotide interconversion


In [336]:
dko_one_drug_path.Pathways = dko_one_drug_path.Pathways.apply(lambda x:x.split(';'))
dko_one_drug_path_ = explode(dko_one_drug_path,['Pathways'],'')
dko_one_drug_path_ = dko_one_drug_path_.drop_duplicates().reset_index(drop=True)

dko_one_drug_path_.to_csv('KO_data/DKO_Drugs_OneDrug_Pathways_tripartite.csv')


In [337]:
# Add the names of the pathway for the DKO drugs
dko_drug = pd.read_csv('KO_data/DKO_Drug_Repurposing_Reduced.csv',index_col=0)
dko_drug.Gene_Pair = dko_drug.Gene_Pair.apply(lambda x:str(x).replace('{','').replace('}','').replace(', ',';').replace("'",''))
dko_drug.Gene_Pair = dko_drug.Gene_Pair.apply(lambda x:x.split(';'))
dko_drug_ =  explode(dko_drug,['Gene_Pair'], fill_value='')
dko_drug_.head()

,Drugs1,Drugs2,Number_Genes,Phenotype_Essentiality,Phenotype_Safety,Gene_Pair
0,Imexon,Valaciclovir,2,10.0,0.0,GUK1
1,Imexon,Valaciclovir,2,10.0,0.0,RRM1
2,Imexon,Valaciclovir,2,10.0,0.0,GUK1
3,Imexon,Valaciclovir,2,10.0,0.0,RRM2
4,Imexon,Acyclovir,2,10.0,0.0,GUK1


In [338]:
genes =  dko_drug_.Gene_Pair.unique()
dko_drug_['Pathways'] = 0
for gene in genes:
    gene_pathways = recon_rules.loc[recon_rules.Genes ==gene,'Pathway'].values.tolist()
    dko_drug_.loc[dko_drug_.Gene_Pair ==gene,'Pathways'] = ';'.join(gene_pathways)
dko_drug_.head()

,Drugs1,Drugs2,Number_Genes,Phenotype_Essentiality,Phenotype_Safety,Gene_Pair,Pathways
0,Imexon,Valaciclovir,2,10.0,0.0,GUK1,Nucleotide interconversion;Nucleotide intercon...
1,Imexon,Valaciclovir,2,10.0,0.0,RRM1,Nucleotide interconversion;Nucleotide intercon...
2,Imexon,Valaciclovir,2,10.0,0.0,GUK1,Nucleotide interconversion;Nucleotide intercon...
3,Imexon,Valaciclovir,2,10.0,0.0,RRM2,Nucleotide interconversion;Nucleotide intercon...
4,Imexon,Acyclovir,2,10.0,0.0,GUK1,Nucleotide interconversion;Nucleotide intercon...


In [339]:
dko_drug_path = dko_drug_.loc[dko_drug_.Pathways.apply(lambda x:len(x))>0,:]
dko_drug_path = dko_drug_path.drop_duplicates().reset_index(drop=True)
dko_drug_path.to_csv('KO_data/DKO_Drug_Repurposing_Reduced_Pathways.csv')
dko_drug_path.head(5)

,Drugs1,Drugs2,Number_Genes,Phenotype_Essentiality,Phenotype_Safety,Gene_Pair,Pathways
0,Imexon,Valaciclovir,2,10.0,0.0,GUK1,Nucleotide interconversion;Nucleotide intercon...
1,Imexon,Valaciclovir,2,10.0,0.0,RRM1,Nucleotide interconversion;Nucleotide intercon...
2,Imexon,Valaciclovir,2,10.0,0.0,RRM2,Nucleotide interconversion;Nucleotide intercon...
3,Imexon,Acyclovir,2,10.0,0.0,GUK1,Nucleotide interconversion;Nucleotide intercon...
4,Imexon,Acyclovir,2,10.0,0.0,RRM1,Nucleotide interconversion;Nucleotide intercon...


In [340]:
dko_drug_path.Pathways = dko_drug_path.Pathways.apply(lambda x:x.split(';'))
dko_drug_path_ = explode(dko_drug_path,['Pathways'],'')
dko_drug_path_ = dko_drug_path_.drop_duplicates().reset_index(drop=True)
dko_drug_path_.to_csv('KO_data/DKO_Drug_Repurposing_Reduced_Pathways_tripartite.csv')


In [341]:
sko_drugs = pd.read_csv('KO_data/SKO_db_Drugs_Unique.csv')
sko_drugs.Gene_Symbol = sko_drugs.Gene_Symbol.apply(lambda x:str(x[2:len(x)-2]))
genes =  sko_drugs.Gene_Symbol.unique().tolist()

In [342]:
import gseapy as gp

enr = gp.enrichr(gene_list=genes,#dko_all_genes,
                 gene_sets=['KEGG_2019_Human'],
                 cutoff=1 # test dataset, use lower value from range(0,1)
                )

In [343]:
enrich = enr.results
enrich = enrich.loc[:,['Term','Genes','P-value','Overlap']]

In [344]:
[x for x in enrich.Genes if x in genes ]

['DHFR', 'SLC7A11', 'SLC7A5', 'ISYNA1', 'CMPK1', 'GUK1', 'SLC7A5']

In [345]:
[x for x in dko_enrichr_.Genes if x in genes ]

['GUK1', 'CMPK1', 'CMPK1', 'SLC7A11', 'SLC7A5', 'SLC7A5', 'ISYNA1']

In [346]:
## Map SKO drugs to drug-gene-pathway
sko_drugs = pd.read_csv('KO_data/SKO_db_Drugs_Unique.csv')
sko_drugs.Gene_Symbol = sko_drugs.Gene_Symbol.apply(lambda x:str(x[2:len(x)-2]).split("""', '"""))#list(str(x[2:len(x)-2]))#.replace("""', '""",';')
sko_drugs = explode(sko_drugs,['Gene_Symbol'],'')

genes =  sko_drugs.Gene_Symbol.unique().tolist()
sko_drugs['Pathways'] = 0
for gene in genes:
    gene_pathways = recon_rules.loc[recon_rules.Genes ==gene,'Pathway'].values.tolist()
    sko_drugs.loc[sko_drugs.Gene_Symbol ==gene,'Pathways'] = ';'.join(gene_pathways)
sko_drugs.to_csv('KO_data/SKO_db_Drugs_Unique_Pathways.csv')
sko_drugs.head(2)

,Drugs,Drugs_Categories,Drugs_Group,Number_Genes,Phenotype_Essentiality,Phenotype_Safety,side_effect_name,Gene_Symbol,Pathways
0,Valaciclovir,"Acyclovir and prodrug|Amino Acids|Amino Acids,...",['approved|investigational'],1,16.0,3.0,[nan],GUK1,Nucleotide interconversion;Nucleotide intercon...
1,Acyclovir,Acyclovir and prodrug|Anti-Infective Agents|An...,['approved'],1,16.0,3.0,[nan],GUK1,Nucleotide interconversion;Nucleotide intercon...


In [347]:
sko_drugs.Pathways = sko_drugs.Pathways.apply(lambda x:x.split(';'))
sko_drugs = sko_drugs.loc[:,['Drugs','Gene_Symbol','Pathways','Phenotype_Safety','Phenotype_Essentiality']]
sko_drugs_ = explode(sko_drugs,['Pathways'],'')
sko_drugs_ = sko_drugs_.loc[sko_drugs_.Pathways.apply(lambda x:len(x))>0,:]
sko_drugs_ = sko_drugs_.drop_duplicates().reset_index(drop=True)
sko_drugs_.to_csv('KO_data/SKO_db_Drugs_Unique_Pathways_tripartite.csv')
sko_drugs_.head(2)

,Drugs,Gene_Symbol,Phenotype_Essentiality,Phenotype_Safety,Pathways
0,Valaciclovir,GUK1,16.0,3.0,Nucleotide interconversion
1,Acyclovir,GUK1,16.0,3.0,Nucleotide interconversion
